# Unit 4 Extending Agent Capabilities with Built-In and Custom Tools

## Introduction and Lesson Overview

Welcome back\! In the last lesson, you learned how to manage multi-turn conversations with your agent, making it possible for your applications to remember and build on previous exchanges. This is a key feature for creating natural, interactive experiences. Now, you are ready to take the next step: extending your agent’s capabilities beyond just generating text.

In real-world applications, agents often need to do more than just talk — they might need to search the web, perform calculations, or interact with external systems. The OpenAI Agents SDK makes this possible by allowing you to add “tools” to your agents. Tools can be built-in (like web search) or custom Python functions that you define yourself. By the end of this lesson, you will know how to add both types of tools to your agent, making it much more powerful and useful.

## Overview of Built-In Tools

The OpenAI Agents SDK comes with several built-in tools that you can add to your agent with just a few lines of code. One of the most useful is the `WebSearchTool`, which allows your agent to search the web for up-to-date information. This is especially helpful when your agent needs to answer questions about recent events or look up facts that are not part of its training data.

Let’s look at a simple example. Suppose you want to create an agent that can answer questions by searching the web. You can do this by adding the `WebSearchTool` to the `tools` parameter of your agent:

```python
from agents import Agent, WebSearchTool

agent = Agent(
    name="Research Assistant",
    instructions="You are a research assistant who can search the web to answer questions.",
    model="gpt-4.1",
    tools=[WebSearchTool()]
)
```

With this setup, your agent can now use the web search tool whenever it needs to find information. This is a big step up from a basic text-only agent, as it can now provide answers based on the latest information available online.

## Creating Custom Function Tools

While built-in tools are powerful, sometimes you need your agent to perform a specific task that is unique to your application. This is where **custom function tools** come in. The OpenAI Agents SDK allows you to turn any Python function into a tool that your agent can use by decorating it with `@function_tool`.

To ensure the agent knows how to call your function tool correctly, you define the expected input parameters using a `TypedDict` class. `TypedDict`s specify the structure and types of the input arguments, making it clear to both the SDK and the agent what data is required. This explicit schema helps the agent construct valid calls to your function tool during its reasoning process.

For example, suppose you want your agent to calculate the number of years between two historical events. You can define a custom function tool like this:

```python
from typing_extensions import TypedDict
from agents import function_tool

# Define a TypedDict for the input parameters
class Years(TypedDict):
    year1: int
    year2: int

# Create the function tool using the decorator
@function_tool
def calculate_years_between(years: Years) -> int:
    """Calculate the absolute difference in years between two years.
    Args:
        years: The two years to calculate the difference between.
    """
    return abs(years["year2"] - years["year1"])
```

Here, the `Years` TypedDict defines that the function expects a dictionary with two integer fields: `year1` and `year2`. The `@function_tool` decorator registers the function as a tool, and the docstring provides a human-readable description that the agent can use to decide when and how to use the tool. By using TypedDicts, you ensure that the agent understands the required input structure, which is essential for reliable and accurate tool usage.

## Integrating Tools With An Agent

Now that you have both a built-in tool (`WebSearchTool`) and a custom function tool (`calculate_years_between`), you can combine them in a single agent. This allows your agent to use both tools as needed to answer user questions.

Here’s how you can create an agent with multiple tools:

```python
from agents import Agent, WebSearchTool

agent = Agent(
    name="Historical Calculator",
    instructions=(
        "You are a helpful assistant that can calculate the number of years "
        "between two historical events. You should first search the web to find "
        "the years of the events, then use the calculation tool to find the "
        "difference."
    ),
    model="gpt-4.1",
    tools=[WebSearchTool(), calculate_years_between]
)
```

In this example, the agent is given clear instructions on how to use its tools: first, search the web to find the years of the events, and then use the calculation tool to find the difference. By listing both tools in the `tools` parameter, you make them available for the agent to use during its reasoning process.

## Verifying Your Custom Function Tools

If you want to verify that the custom function tool you created is correctly set up for your agent to use, you can inspect the agent’s list of tools and print out details for each function tool. This is especially useful for checking that your tool’s name, description, and parameter schema are registered as expected.

To do this, you can iterate through the agent’s `tools` list and print information only for tools that are instances of `FunctionTool` (which includes your custom function tools, but not built-in tools like `WebSearchTool`):

```python
import json
from agents import FunctionTool

# List only the custom function tools available to the agent
for tool in agent.tools:
    if isinstance(tool, FunctionTool):
        print(f"Tool: {tool.name}")
        print(f"Description: {tool.description}")
        print(f"Params:\n{json.dumps(tool.params_json_schema, indent=2)}\n")
```

This will output details for each function tool, allowing you to confirm that your custom tool is properly registered and ready for the agent to use.

```
Tool: calculate_years_between
Description: Calculate the absolute difference in years between two years.

Params:
{
  "$defs": {
    "Years": {
      "properties": {
        "year1": {
          "title": "Year1",
          "type": "integer"
        },
        "year2": {
          "title": "Year2",
          "type": "integer"
        }
      },
      "required": [
        "year1",
        "year2"
      ],
      "title": "Years",
      "type": "object",
      "additionalProperties": false
    }
  },
  "properties": {
    "years": {
      "description": "The two years to calculate the difference between.",
      "properties": {
        "year1": {
          "title": "Year1",
          "type": "integer"
        },
        "year2": {
          "title": "Year2",
          "type": "integer"
        }
      },
      "required": [
        "year1",
        "year2"
      ],
      "title": "Years",
      "type": "object",
      "additionalProperties": false
    }
  },
  "required": [
    "years"
  ],
  "title": "calculate_years_between_args",
  "type": "object",
  "additionalProperties": false
}
```

## Running The Agent With Tools In Action

Once your agent has its tools, you can run it just like before. The difference is that now, the agent can decide when to use each tool to answer the user’s question. For example, if you ask, “How many years are there between Benjamin Franklin’s discovery of electricity and ChatGPT?”, the agent will first use the web search tool to find the years of each event, then use the calculation tool to compute the difference.

Here’s how you can run the agent and see the result:

```python
import asyncio
import json
from agents import Runner

async def main():
    result = await Runner.run(
        starting_agent=agent,
        input="How many years between Benjamin Franklin's discovery of electricity and ChatGPT?"
    )

    # Print the result list with indentation
    print(json.dumps(result.to_input_list(), indent=2))

if __name__ == "__main__":
    asyncio.run(main())
```

By using the `to_input_list()` method on the result, you can view a detailed, step-by-step breakdown of how the agent used its tools—including each tool invocation, the arguments passed, and the outputs returned. The output might look something like this:

```
[
  {
    "content": "How many years between Benjamin Franklin's discovery of electricity and ChatGPT?",
    "role": "user"
  },
  {
    "id": "ws_6810f534dc1c8192a613868e2cf5b34508d928afdab1f63e",
    "status": "completed",
    "type": "web_search_call"
  },
  {
    "id": "msg_6810f5389ef481928c308f94c155fc8c08d928afdab1f63e",
    "content": [
      {
        "annotations": [
          {
            "end_index": 231,
            "start_index": 124,
            "title": "The Kite Experiment, 19 October 1752",
            "type": "url_citation",
            "url": "https://founders.archives.gov/documents/Franklin/01-04-02-0135?utm_source=openai"
          },
          {
            "end_index": 356,
            "start_index": 281,
            "title": "What is ChatGPT? Everything you need to know",
            "type": "url_citation",
            "url": "https://www.tomsguide.com/news/chatgpt?utm_source=openai"
          }
        ],
        "text": "Benjamin Franklin conducted his famous kite experiment in June 1752, demonstrating that lightning is a form of electricity. ([founders.archives.gov](https://founders.archives.gov/documents/Franklin/01-04-02-0135?utm_source=openai)) ChatGPT was launched by OpenAI in November 2022. ([tomsguide.com](https://www.tomsguide.com/news/chatgpt?utm_source=openai)) Therefore, there are 270 years between these two events. ",
        "type": "output_text"
      }
    ],
    "role": "assistant",
    "status": "completed",
    "type": "message"
  },
  {
    "arguments": "{\"years\":{\"year1\":1752,\"year2\":2022}}",
    "call_id": "call_EhtsnJ85Luo907LNesxuP8RX",
    "name": "calculate_years_between",
    "type": "function_call",
    "id": "fc_6810f53c830881928f77f70389fd11bf08d928afdab1f63e",
    "status": "completed"
  },
  {
    "call_id": "call_EhtsnJ85Luo907LNesxuP8RX",
    "output": "270",
    "type": "function_call_output"
  },
  {
    "id": "msg_6810f53e1a7881928947b75b2fa60cd308d928afdab1f63e",
    "content": [
      {
        "annotations": [],
        "text": "There are 270 years between Benjamin Franklin's discovery of electricity in 1752 and the launch of ChatGPT in 2022.",
        "type": "output_text"
      }
    ],
    "role": "assistant",
    "status": "completed",
    "type": "message"
  }
]
```

This shows how the agent used both tools to answer the question, providing a clear and accurate response.

## The Challenge of Tool Integration and the Promise of MCP

Integrating tools with different agentic systems can be difficult and time-consuming. Each platform often has its own way of defining and connecting tools, leading to duplicated work, inconsistent APIs, and maintenance headaches. As you add more tools or want to support multiple agent frameworks, these challenges multiply.

The **Model Context Protocol (MCP)** is a game changer. MCP is an open standard that lets you develop a tool once and make it available to any agentic system that supports the protocol. It standardizes how tools are described, discovered, and invoked, making integration much simpler and more reliable. With MCP, you can build tools that work across platforms, reduce integration effort, and improve security and governance.

In the next course, you’ll learn what MCP is, how it works, and how to develop and integrate your own MCP server—unlocking even more powerful and flexible agentic applications.

## Summary and Preparing For Practice Exercises

In this lesson, you learned how to extend your agent’s capabilities by adding both built-in and custom function tools using the OpenAI Agents SDK. You saw how to use the `WebSearchTool` for real-time information and how to create your own function tools with the `@function_tool` decorator and TypedDicts for clear input schemas. You also learned how to combine multiple tools in a single agent, verify tool registration, and observe how agents use these tools step by step.

You also explored the broader challenges of integrating tools across different agentic systems, and how the Model Context Protocol (MCP) is emerging as a solution to these problems. In the next course, you’ll dive deeper into MCP—learning what it is, how it works, and how to develop and integrate your own MCP server to make your tools universally accessible to any agentic platform.

Now, get ready to put your knowledge into practice with a set of hands-on exercises. These will help you reinforce what you’ve learned by building and using agents with both built-in and custom tools.

## Adding a Built-In Tool to Your Agent

Now it’s your turn to put built-in tools into practice by setting up an agent that can search the web for answers.

Your task is to create an agent called Research Assistant that uses the WebSearchTool. This tool will let your agent look up real-time information online. To complete this exercise:

Add the WebSearchTool to the agent’s tools list.
Run the agent with a research question asking for the latest information about something.
Once you’ve completed these steps, the code will print the result's final output. This will demonstrate how easy it is to extend your agent’s abilities with just a few lines of code.

```python
import asyncio
from agents import Agent, WebSearchTool, Runner

# Create an agent with the WebSearchTool
agent = Agent(
    name="Research Assistant",
    instructions="You are a research assistant who can search the web to answer questions.",
    model="gpt-4.1",
    # TODO: Add the WebSearchTool to the tools list
)


async def main():
    # TODO: Run the agent with a research question asking about latest information
    result = await Runner.run(
        starting_agent=agent,
        input=
    )

    # Print the final output
    print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

```

```python
import asyncio
from agents import Agent, WebSearchTool, Runner

# Create an agent with the WebSearchTool
agent = Agent(
    name="Research Assistant",
    instructions="You are a research assistant who can search the web to answer questions.",
    model="gpt-4.1",
    # Add the WebSearchTool to the tools list
    tools=[WebSearchTool()]
)


async def main():
    # Run the agent with a research question asking about latest information
    result = await Runner.run(
        starting_agent=agent,
        input="What are the latest advancements in AI in 2025?"
    )

    # Print the final output
    print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

```

## Exploring Agent Reasoning Steps

Now, let’s take a closer look at how the agent actually uses its tools step by step.

Your task is to update your code so that, instead of only printing the final answer, it prints the full sequence of steps the agent took to answer your research question.

Change the print statement so it uses the to_input_list() method and formats the output with json.dumps() and indentation. This will show you the detailed process, not just the final answer.

This will help you see exactly when and how the WebSearchTool was used.


```python
import asyncio
import json
from agents import Agent, WebSearchTool, Runner

# Create an agent with the WebSearchTool
agent = Agent(
    name="Research Assistant",
    instructions="You are a research assistant who can search the web to answer questions.",
    model="gpt-4.1",
    tools=[WebSearchTool()]
)


async def main():
    # Run the agent with a research question
    result = await Runner.run(
        starting_agent=agent,
        input="What are the latest discoveries in renewable energy?"
    )

    # TODO: Print the detailed step-by-step result list using to_input_list() and json.dumps() with indentation
    print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

```

```python
import asyncio
import json
from agents import Agent, WebSearchTool, Runner

# Create an agent with the WebSearchTool
agent = Agent(
    name="Research Assistant",
    instructions="You are a research assistant who can search the web to answer questions.",
    model="gpt-4.1",
    tools=[WebSearchTool()]
)


async def main():
    # Run the agent with a research question
    result = await Runner.run(
        starting_agent=agent,
        input="What are the latest discoveries in renewable energy?"
    )

    # Print the detailed step-by-step result list using to_input_list() and json.dumps() with indentation
    print(json.dumps(result.to_input_list(), indent=2))

if __name__ == "__main__":
    asyncio.run(main())
```

## Build a Custom Calculation Tool

You’ve just seen how to add a built-in tool to your agent, giving it the power to search the web for up-to-date information. Now, let’s take things a step further by creating your own custom function tool.

In this exercise, you’ll practice creating a custom function tool and making it available to your agent. This tool will help your agent calculate the number of years between two events, given as numbers. Your task is to:

Add the @function_tool decorator to the calculate_years_between function so that it becomes available as a tool.
Complete the function so it returns the absolute difference between the two years provided in the years dictionary.
Add your calculate_years_between function to the agent’s tools list.
When you run the code, your agent will be able to answer questions like “How many years between 1990 and 2020?” by using your custom calculation tool.

```python
import asyncio
import json
from typing_extensions import TypedDict
from agents import Agent, Runner, function_tool


# Define a type for the years received by the function tool
class Years(TypedDict):
    year1: int
    year2: int


# TODO: Add the @function_tool decorator here
def calculate_years_between(years: Years) -> int:
    """Calculate the absolute difference in years between two years.

    Args:
        years: The two years to calculate the difference between.
    """
    # TODO: Return the absolute difference between years["year2"] and years["year1"]


# Create an agent with the custom function tool
agent = Agent(
    name="Year Difference Calculator",
    instructions=(
        "You are a helpful assistant that can calculate the number of years "
        "between two events when given their years. Always use the available tool "
        "to perform the calculation instead of doing the math yourself."
    ),
    model="gpt-4.1",
    # TODO: Add calculate_years_between to the tools list
    tools=[]
)


async def main():
    # Run the agent with a query
    result = await Runner.run(
        starting_agent=agent,
        input="How many years between 1990 and 2020?"
    )

    # Print the result list
    print(json.dumps(result.to_input_list(), indent=2))

if __name__ == "__main__":
    asyncio.run(main())

```

```python
import asyncio
import json
from typing_extensions import TypedDict
from agents import Agent, Runner, function_tool


# Define a type for the years received by the function tool
class Years(TypedDict):
    year1: int
    year2: int


# Add the @function_tool decorator here
@function_tool
def calculate_years_between(years: Years) -> int:
    """Calculate the absolute difference in years between two years.

    Args:
        years: The two years to calculate the difference between.
    """
    # Return the absolute difference between years["year2"] and years["year1"]
    return abs(years["year2"] - years["year1"])


# Create an agent with the custom function tool
agent = Agent(
    name="Year Difference Calculator",
    instructions=(
        "You are a helpful assistant that can calculate the number of years "
        "between two events when given their years. Always use the available tool "
        "to perform the calculation instead of doing the math yourself."
    ),
    model="gpt-4.1",
    # Add calculate_years_between to the tools list
    tools=[calculate_years_between]
)


async def main():
    # Run the agent with a query
    result = await Runner.run(
        starting_agent=agent,
        input="How many years between 1990 and 2020?"
    )

    # Print the result list
    print(json.dumps(result.to_input_list(), indent=2))

if __name__ == "__main__":
    asyncio.run(main())
```

## Exploring Tool Metadata in Agents

You’ve just practiced building a custom function tool and adding it to your agent. Now, let’s see how the way you define your tool in Python shapes how the agent understands and describes it.

In this exercise, you will:

Update the parameter names in both the Years TypedDict and the calculate_years_between function from year1 and year2 to year_one and year_two.
Make sure the function uses the new parameter names everywhere they are needed.
Add code to loop through the agent’s tools, filter for function tools, and print out each tool’s name, description, and JSON parameter schema.
This will help you see how changes in your Python code are automatically reflected in the agent’s tool metadata. It’s a great way to understand the connection between your function definitions and what the agent can do.

```python
import json
from typing_extensions import TypedDict
from agents import Agent, FunctionTool, function_tool


# Define a type for the years received by the function tool
class Years(TypedDict):
    # TODO: Change 'year1' to 'year_one'
    year1: int
    # TODO: Change 'year2' to 'year_two'
    year2: int


# TODO: Update the parameter names in the function implementation
@function_tool
def calculate_years_between(years: Years) -> int:
    """Calculate the absolute difference in years between two years.

    Args:
        years: The two years to calculate the difference between.
    """
    return abs(years["year2"] - years["year1"])


# Create an agent with the custom function tool
agent = Agent(
    name="Year Difference Calculator",
    instructions=(
        "You are a helpful assistant that can calculate the number of years "
        "between two events when given their years. Always use the available tool "
        "to perform the calculation instead of doing the math yourself."
    ),
    model="gpt-4.1",
    tools=[calculate_years_between]
)

# TODO: Print the function tool metadata (name, description, params) for each function tool in agent.tools

```

```python
import json
from typing_extensions import TypedDict
from agents import Agent, FunctionTool, function_tool


# Define a type for the years received by the function tool
class Years(TypedDict):
    # Change 'year1' to 'year_one'
    year_one: int
    # Change 'year2' to 'year_two'
    year_two: int


# Update the parameter names in the function implementation
@function_tool
def calculate_years_between(years: Years) -> int:
    """Calculate the absolute difference in years between two years.

    Args:
        years: The two years to calculate the difference between.
    """
    return abs(years["year_two"] - years["year_one"])


# Create an agent with the custom function tool
agent = Agent(
    name="Year Difference Calculator",
    instructions=(
        "You are a helpful assistant that can calculate the number of years "
        "between two events when given their years. Always use the available tool "
        "to perform the calculation instead of doing the math yourself."
    ),
    model="gpt-4.1",
    tools=[calculate_years_between]
)

# Print the function tool metadata (name, description, params) for each function tool in agent.tools
for tool in agent.tools:
    if isinstance(tool, FunctionTool):
        print(f"Tool: {tool.name}")
        print(f"Description: {tool.description}")
        print(f"Params:\n{json.dumps(tool.params_json_schema, indent=2)}\n")

```

## Combining Tools for Smarter Agents

Now it’s time to bring these skills together by building an agent that can use both types of tools in a single workflow.

Your task is to create an agent that can answer questions about the number of years between two historical events — even if it needs to look up the years first. To do this, you will:

Make sure the agent has access to both the WebSearchTool and your custom calculate_years_between function tool.
Write clear instructions for the agent so it knows to first search for the years of the events, then use the calculation tool to find the difference.
Run the agent with a question that requires both steps, such as asking about the years between two inventions.
Observe the result so you can see how the agent uses both tools in sequence. This exercise will help you see how agents can combine multiple tools to solve more complex problems.


```python
import asyncio
import json
from typing_extensions import TypedDict
from agents import Agent, Runner, WebSearchTool, function_tool


# Define a type for the years received by the function tool
class Years(TypedDict):
    year1: int
    year2: int


# Register the function as a tool
@function_tool
def calculate_years_between(years: Years) -> int:
    """Calculate the absolute difference in years between two years.

    Args:
        years: The two years to calculate the difference between.
    """
    return abs(years["year2"] - years["year1"])


# TODO: Create an agent with both the web search tool and the custom calculation tool
agent = Agent(
    name="Historical Calculator",
    # TODO: Complete the instructions to tell the agent to use both tools
    instructions="",
    model="gpt-4.1",
    # TODO: Add both WebSearchTool() and calculate_years_between to the tools list
    tools=[]
)


async def main():
    # TODO: Run the agent with a query that requires both tools
    result = await Runner.run(
        starting_agent=agent,
        input=
    )

    # Print the result list
    print("Input List:")
    print(json.dumps(result.to_input_list(), indent=2))
    
    # Also print the final output
    print("\nFinal Output:")
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

```

```python
import asyncio
import json
from typing_extensions import TypedDict
from agents import Agent, Runner, WebSearchTool, function_tool


# Define a type for the years received by the function tool
class Years(TypedDict):
    year1: int
    year2: int


# Register the function as a tool
@function_tool
def calculate_years_between(years: Years) -> int:
    """Calculate the absolute difference in years between two years.

    Args:
        years: The two years to calculate the difference between.
    """
    return abs(years["year2"] - years["year1"])


# Create an agent with both the web search tool and the custom calculation tool
agent = Agent(
    name="Historical Calculator",
    # Complete the instructions to tell the agent to use both tools
    instructions="You are an agent that can determine the number of years between two historical events. First, use the WebSearchTool to find the years of the events. Then, use the calculate_years_between function tool to find the absolute difference between those two years and provide the final answer.",
    model="gpt-4.1",
    # Add both WebSearchTool() and calculate_years_between to the tools list
    tools=[WebSearchTool(), calculate_years_between]
)


async def main():
    # Run the agent with a query that requires both tools
    result = await Runner.run(
        starting_agent=agent,
        input="How many years were there between the invention of the telephone and the invention of the internet?"
    )

    # Print the result list
    print("Input List:")
    print(json.dumps(result.to_input_list(), indent=2))
    
    # Also print the final output
    print("\nFinal Output:")
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

```